## Draggable Label

`DraggbelLabel` is a label that can be dragged and dropped to other fields.

In [1]:
from ipywidgets import DraggableLabel, Textarea
l = DraggableLabel("Drag me", draggable=False)
l.draggable = True
l

DraggableLabel(value='Drag me', draggable=True)

You can drag this label anywhere (could be your shell etc.), but also to a text area:

In [2]:
Textarea()

Textarea(value='')

## `on_drop` handler

`DraggableLabel` can also become the drop zone (you can drop other stuff on it), if you implement the `on_drop` handler.

In [3]:
from ipywidgets import DraggableLabel
l1 = DraggableLabel("Drag me", draggable=True)
l1

DraggableLabel(value='Drag me', draggable=True)

Now, drag this label on the label below.

In [4]:
l2 = DraggableLabel("Drop on me")
def on_drop_handler(widget, data):
    """"Arguments:
    
    widget : widget class
        widget on which something was dropped
        
    data : dict
        extra data sent from the dragged widget"""
    
    text = data['text/plain']
    widget.value = "congrats, you dropped '{}'".format(text)
l2.on_drop(on_drop_handler)
l2

DraggableLabel(value='Drop on me')

**Note** : You can also drop other stuff (like files, images, links, selections, etc). Try it out!

## Adding elements to the container

You can drag widgets to Box container and its subclasses by simply dragging.

In [19]:
DraggableLabel("Drag me 1", draggable=True)

DraggableLabel(value='Drag me', draggable=True)

In [22]:
DraggableLabel("Drag me 2", draggable=True)

DraggableLabel(value='Drag me 2', draggable=True)

In [23]:
from ipywidgets import Box, DraggableLabel, Layout, Label, VBox
normal_box = Box([Label('drag labels here')],
                 layout=Layout(width='400px', height='100px'),
                 dropzone=True)
normal_box

Box(children=(Label(value='drag labels here'),), dropzone=True, layout=Layout(height='100px', width='400px'))

It also works for Box subclasses:

In [21]:
vbox = VBox([Label('drag labels here')],
                 layout=Layout(width='200px', height='100px'),
                 dropzone=True)
vbox

## DropBox supports arbitrary drop operations

If you have more specific needs for the drop behaviour you can also use DropBox widgets, which implements `on_drop` handlers.

This DropBox will replace elements with text of the dropped element (works also for stuff which is not widget):

In [27]:
from ipywidgets import DropBox, DraggableLabel, Layout, Button

label = DraggableLabel("Drag me", draggable=True)
label

DraggableLabel(value='Drag me', draggable=True)

or **Select and drag me!**

In [30]:
box = DropBox([Label("Drop here!")],
              layout=Layout(width='200px', height='100px'))
def on_drop(widget, data):
    text = data['text/plain']
    widget.children = (Button(description=text.upper()), )
box.on_drop(on_drop)
box

DropBox(children=(Label(value='Drop here!'),), layout=Layout(height='100px', width='200px'))

## Adding widgets to container with handler

You can also reproduce the Box example (adding elements to Box container) using `DropBox` with a custom handler:

In [31]:
from ipywidgets import DropBox, DraggableLabel, Layout, Label

label = DraggableLabel("Drag me", draggable=True)
label

DraggableLabel(value='Drag me', draggable=True)

In [11]:
box = DropBox([Label('Drop here')], 
              layout=Layout(width='200px', height='100px'))
def on_drop(widget, data):
    source = widget.widgets[data['application/x-widget']]
    widget.children += (source, )
box.on_drop(on_drop)
box

DropBox(children=(Label(value='Drop here'),), layout=Layout(height='100px', width='200px'))

**Explanation**: Label widget sets data on the drop event of type `application/x-widget` that contains the widget id of the label.

## Setting custom data on the drop event

You can also set custom data on `DraggableLabel` that can be retreived and used in `on_drop` event.

In [12]:
from ipywidgets import DraggableLabel
l = DraggableLabel("Drag me", draggable=True)
l.drag_data = {'application/custom-data' : 'Custom data'}
l

DraggableLabel(value='Drag me', drag_data={'application/custom-data': 'Custom data'}, draggable=True)

In [13]:
l2 = DraggableLabel("Drop here")
def on_drop_handler(widget, data):
    """"Arguments:
    
    widget : widget class
        widget on which something was dropped
        
    data : dict
        extra data sent from the dragged widget"""
    
    text = data['text/plain']
    widget_id = data['application/x-widget']
    custom_data = data['application/custom-data']
    widget.value = ("you dropped widget ID '{}...' "
                    "with text '{}' and custom data '{}'"
                   ).format(widget_id[:5], text,  custom_data)
l2.on_drop(on_drop_handler)
l2

DraggableLabel(value='Drop here')

## Draggable data columns

This implements a bit more complex example.

In [14]:
import bqplot.pyplot as plt
from ipywidgets import Label, GridspecLayout, DropBox, Layout, DraggableLabel
import json

In [15]:
def create_table(data):
    n_cols = len(data)
    n_rows = max(len(column) for column in data.values())
    grid = GridspecLayout(n_rows+1, n_cols)
    columnames = list(data.keys())
    for i in range(n_cols):
        column = columnames[i]
        data_json = json.dumps(data[column])
        grid[0, i] = DraggableLabel(columnames[i], draggable=True)
        grid[0, i].draggable = True
        grid[0, i].drag_data = {'data/app' : data_json}
        for j in range(n_rows):
            grid[j+1, i] = DraggableLabel(str(data[column][j]), draggable=True)
            grid[j+1, i].drag_data = {'data/app' : data_json}
    return grid

In [16]:
box = DropBox([Label("Drag data from the table and drop it here.")], layout=Layout(height='500px', width='800px'))
def box_ondrop(widget, data):
    fig = plt.figure()
    y = json.loads(data['data/app'])
    plt.plot(y)
    widget.children = (fig, )
box.on_drop(box_ondrop)

In [17]:
plot_data = {'col1' : [4, 2, 1],
             'col2' : [1, 3 ,4],
             'col3' : [-1, 2, -3]}
create_table(plot_data)

GridspecLayout(children=(DraggableLabel(value='col1', drag_data={'data/app': '[4, 2, 1]'}, draggable=True, lay…

In [18]:
box

DropBox(children=(Label(value='Drag data from the table and drop it here.'),), layout=Layout(height='500px', w…